In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

In [4]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [5]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [8]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [9]:
def generator(noise_z):
    hidden = tf.nn.relu(
        tf.matmul(noise_z, G_W1 + G_b1)
    )
    output = tf.nn.sigmoid(
        tf.matmul(hidden, G_W2 + G_b2)
    )
    return output

In [10]:
def discriminator(inputs):
    hidden = tf.nn.relu(
        tf.matmul(inputs, D_W1) + D_b1
    )
    output = tf.nn.sigmoid(
        tf.matmul(hidden, D_W2) + D_b2
    )
    
    return output

In [11]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [12]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [13]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

In [14]:
loss_G = tf.reduce_mean(tf.log(D_gene))

In [15]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [16]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [30]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

In [31]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
        
    print('Epoch: %4d' % (epoch + 1), 'D Loss: %.4f' % loss_val_D, 'G Loss: %.4f' % loss_val_G)
    
    if epoch == 0 or (epoch + 1) % 10 ==0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)
print("Done Learning")

Epoch:    1 D Loss: -1.4334 G Loss: -1.3852
Epoch:    2 D Loss: -0.3816 G Loss: -2.1373
Epoch:    3 D Loss: -0.6862 G Loss: -1.3352
Epoch:    4 D Loss: -0.3589 G Loss: -1.7184
Epoch:    5 D Loss: -0.4419 G Loss: -2.1491
Epoch:    6 D Loss: -0.2950 G Loss: -2.5336
Epoch:    7 D Loss: -0.1443 G Loss: -2.9088
Epoch:    8 D Loss: -0.3079 G Loss: -2.5775
Epoch:    9 D Loss: -0.2484 G Loss: -2.8398
Epoch:   10 D Loss: -0.1399 G Loss: -2.9747
Epoch:   11 D Loss: -0.2242 G Loss: -3.0732
Epoch:   12 D Loss: -0.1868 G Loss: -3.0094
Epoch:   13 D Loss: -0.3015 G Loss: -3.0375
Epoch:   14 D Loss: -0.2287 G Loss: -3.2606
Epoch:   15 D Loss: -0.4614 G Loss: -2.8101
Epoch:   16 D Loss: -0.1432 G Loss: -3.6351
Epoch:   17 D Loss: -0.2587 G Loss: -3.5115
Epoch:   18 D Loss: -0.2002 G Loss: -3.3039
Epoch:   19 D Loss: -0.1791 G Loss: -3.3841
Epoch:   20 D Loss: -0.2663 G Loss: -3.0755
Epoch:   21 D Loss: -0.4377 G Loss: -3.3125
Epoch:   22 D Loss: -0.2159 G Loss: -3.7231
Epoch:   23 D Loss: -0.3446 G Lo

# 원하는 숫자 생성하기 (Generate specific number)

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

total_epoch = 100
batch_size = 100
n_hidden = 256
n_input = 28 * 28
n_noise = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
Z = tf.placeholder(tf.float32, [None, n_noise])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def generator(noise, labels):
    with tf.variable_scope('generator'):
        inputs = tf.concat([noise, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, n_input, activation=tf.nn.sigmoid)
        
    return output

In [4]:
def discriminator(inputs, labels, reuse=None):
    with tf.variable_scope('discriminator') as scope:
        if reuse:
            scope.reuse_variables()
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden, 1, activation=None)
    return output

In [5]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size=[batch_size, n_noise])

In [6]:
G = generator(Z, Y)
D_real = discriminator(X, Y)
D_gene = discriminator(G, Y, True)

In [7]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real, labels=tf.ones_like(D_real)))
loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.zeros_like(D_gene)))
loss_D = loss_D_real + loss_D_gene

In [8]:
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [9]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
train_D = tf.train.AdamOptimizer().minimize(loss_D, var_list=vars_D)
train_G = tf.train.AdamOptimizer().minimize(loss_G, var_list=vars_G)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

In [11]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Y:batch_ys, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Y: batch_ys, Z: noise})
        
    print("Epoch: %4d" % (epoch + 1), "D loss: %.4f" % (loss_val_D), "G loss: %.4f" % (loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Y: mnist.test.labels[:sample_size], Z: noise})
        fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))
        
        for i in range(sample_size):
            ax[0][i].set_axis_off()
            ax[1][i].set_axis_off()
            
            ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
            ax[1][i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('samples2/{}.png'.format(str(epoch + 1).zfill(3)), bbox_inches='tight')
        plt.close(fig)
            
print("최적화 완료")

Epoch:    1 D loss: 0.0065 G loss: 8.0198
Epoch:    2 D loss: 0.0051 G loss: 8.7165
Epoch:    3 D loss: 0.0109 G loss: 8.5445
Epoch:    4 D loss: 0.0033 G loss: 8.8095
Epoch:    5 D loss: 0.0008 G loss: 9.7644
Epoch:    6 D loss: 0.0003 G loss: 9.4126
Epoch:    7 D loss: 0.0001 G loss: 12.0422
Epoch:    8 D loss: 0.0014 G loss: 11.6996
Epoch:    9 D loss: 0.0011 G loss: 9.0091
Epoch:   10 D loss: 0.0007 G loss: 10.4060
Epoch:   11 D loss: 0.0000 G loss: 15.9834
Epoch:   12 D loss: 0.0000 G loss: 13.9789
Epoch:   13 D loss: 0.0000 G loss: 13.8293
Epoch:   14 D loss: 0.0000 G loss: 15.9755
Epoch:   15 D loss: 0.0000 G loss: 16.6407
Epoch:   16 D loss: 0.0000 G loss: 17.3202
Epoch:   17 D loss: 0.0000 G loss: 16.0235
Epoch:   18 D loss: 0.0000 G loss: 17.2140
Epoch:   19 D loss: 0.0000 G loss: 15.7061
Epoch:   20 D loss: 0.0000 G loss: 12.5900
Epoch:   21 D loss: 0.0000 G loss: 14.1879
Epoch:   22 D loss: 0.0000 G loss: 13.5125
Epoch:   23 D loss: 0.0000 G loss: 11.6081
Epoch:   24 D loss